# Finite width effects on the lepton collider in simple collider processes

We are trying to understand the finite-width effects in simple $s$-channel processes like $\mu^+ \mu^- \to Z^\prime \to \ell \bar{\ell}$, where $Z^\prime$ (or any other mediator) has a finite width. \
At lepton colliders, the PDFs are calculable (and are known), so we can use this knowledge to our advantage. The PDFs can be found in https://arxiv.org/pdf/2303.16964 or better here: https://github.com/DavidMarzocca/LePDF.

One of the steps is to compare the analytic forms of the PDFs with the numerical ones, just out of control. We can import the numerical values of the PDFs through LHAPDF and interpolate numerically with scipy.

Note that the LePDF files contain the values of $x \times f(x)$!

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy

from scipy.interpolate import RegularGridInterpolator

import lhapdf
import sympy

Set the folder:

In [ ]:
os.chdir("/home/armanubuntu/finite-width-lepton-colliders")

The PDF functions written in analytic form:


In [ ]:
# Utility functions
def theta(x):
    """Heaviside step function."""
    return np.where(np.array(x) > 0, 1.0, 0.0)

def delta_gaussian(x, sigma=1e-3):
    """Narrow Gaussian approximation to δ(1-x)."""
    x = np.asarray(x)
    norm = 1.0 / (sigma * np.sqrt(2.0 * np.pi))
    return norm * np.exp(-0.5 * ((x - 1.0) / sigma)**2)


# -----------------------------------------------------------------------------
# 1. f_mu^(alpha^2)(x,t)
# -----------------------------------------------------------------------------
def f_mu_alpha2(x, t, alpha_gamma, P_Vff, I_VVf=None, I_ffff=None, delta=delta_gaussian):
    x = np.asarray(x)
    dg = delta(x)
    P = np.asarray(P_Vff(x))
    I_V = np.asarray(I_VVf(x)) if I_VVf is not None else np.zeros_like(x)
    I_f = np.asarray(I_ffff(x)) if I_ffff is not None else np.zeros_like(x)
    pref = alpha_gamma / (2.0 * np.pi)
    term1 = dg
    term2 = pref * t * (1.5 * dg + P)
    term3 = 0.5 * (pref * t)**2 * (2.25 * dg + 3.0 * P + I_V + I_f)
    return term1 + term2 + term3


# -----------------------------------------------------------------------------
# 2. f_lsea^(alpha^2)(x,t)
# -----------------------------------------------------------------------------
def f_lsea_alpha2(x, t, alpha_gamma, I_VVf, delta=delta_gaussian):
    x = np.asarray(x)
    I_V = np.asarray(I_VVf(x))
    pref = alpha_gamma / (2.0 * np.pi)
    return 0.5 * (pref * t)**2 * I_V


# -----------------------------------------------------------------------------
# 3. f_gamma^(alpha^2)(x,t)
# -----------------------------------------------------------------------------
def f_gamma_alpha2(x, t, alpha_gamma, P_Vf, I_Vfff=None, Nf_QED=0):
    x = np.asarray(x)
    P = np.asarray(P_Vf(x))
    Ifff = np.asarray(I_Vfff(x)) if I_Vfff is not None else np.zeros_like(x)
    pref = alpha_gamma / (2.0 * np.pi)
    term1 = pref * t * P
    term2 = 0.5 * (pref * t)**2 * ((1.5 - 2.0/3.0 * Nf_QED) * P + Ifff)
    return term1 + term2


# -----------------------------------------------------------------------------
# 4. f_W±^(alpha)(x,Q^2)
# -----------------------------------------------------------------------------
def f_Wpm_alpha(x, Q2, alpha2, P_Vpm_fL, mW, m_mu):
    x = np.asarray(x)
    P = np.asarray(P_Vpm_fL(x))
    denom = Q2 + (1.0 - x) * mW**2
    denom = np.where(denom == 0, np.finfo(float).tiny, denom)
    logterm = np.log((Q2 + (1.0 - x) * mW**2) / (m_mu**2 + (1.0 - x) * mW**2))
    return (alpha2 / (8.0 * np.pi)) * P * (logterm - Q2 / denom)


# -----------------------------------------------------------------------------
# 5. f_WL^(alpha)(x,Q^2)
# -----------------------------------------------------------------------------
def f_WL_alpha(x, Q2, alpha2, mW):
    x = np.asarray(x)
    safe_x = np.where(x == 0, np.finfo(float).tiny, x)
    denom = Q2 + (1.0 - x) * mW**2
    denom = np.where(denom == 0, np.finfo(float).tiny, denom)
    return (alpha2 / (4.0 * np.pi)) * ((1.0 - x) / safe_x) * (Q2 / denom)


# -----------------------------------------------------------------------------
# 6. f_Z±^(alpha)(x,Q^2)
# -----------------------------------------------------------------------------
def f_Zpm_alpha(x, Q2, alpha2, cW, P_Vpm_fL, P_Vpm_fR, QZ_muL, QZ_muR, mZ, m_mu):
    x = np.asarray(x)
    P_L = np.asarray(P_Vpm_fL(x))
    P_R = np.asarray(P_Vpm_fR(x))
    prefP = P_L * (QZ_muL**2) + P_R * (QZ_muR**2)
    denom = Q2 + (1.0 - x) * mZ**2
    denom = np.where(denom == 0, np.finfo(float).tiny, denom)
    logterm = np.log((Q2 + (1.0 - x) * mZ**2) / (m_mu**2 + (1.0 - x) * mZ**2))
    return (alpha2 / (4.0 * np.pi * cW**2)) * prefP * (logterm - Q2 / denom)


# -----------------------------------------------------------------------------
# 7. f_{Z/gamma ±}^{(alpha)}(x,Q^2)
# -----------------------------------------------------------------------------
def f_Zgamma_pm_alpha(x, Q2, alpha_gamma, alpha2, cW, P_Vpm_fL, P_Vpm_fR, QZ_muL, QZ_muR, mZ, m_mu):
    x = np.asarray(x)
    P_L = np.asarray(P_Vpm_fL(x))
    P_R = np.asarray(P_Vpm_fR(x))
    prefP = P_L * QZ_muL + P_R * QZ_muR
    logterm = np.log((Q2 + (1.0 - x) * mZ**2) / (m_mu**2 + (1.0 - x) * mZ**2))
    return - np.sqrt(alpha_gamma * alpha2) / (2.0 * np.pi * cW) * prefP * logterm


# -----------------------------------------------------------------------------
# 8. f_ZL^(alpha)(x,Q^2)
# -----------------------------------------------------------------------------
def f_ZL_alpha(x, Q2, alpha2, cW, QZ_muL, QZ_muR, mZ):
    x = np.asarray(x)
    safe_x = np.where(x == 0, np.finfo(float).tiny, x)
    denom = Q2 + (1.0 - x) * mZ**2
    denom = np.where(denom == 0, np.finfo(float).tiny, denom)
    prefQ = (QZ_muL**2 + QZ_muR**2)
    return (alpha2 / (2.0 * np.pi * cW**2)) * ((1.0 - x) / safe_x) * prefQ * (Q2 / denom)


# -----------------------------------------------------------------------------
# 9. f_nu_mu^(alpha)(x,Q^2)
# -----------------------------------------------------------------------------
def f_nu_mu_alpha(x, Q2, alpha2, P_Vff, mW):
    x = np.asarray(x)
    P = np.asarray(P_Vff(x))
    threshold = mW**2 / (1.0 - x)**2
    th = theta(Q2 - threshold)
    denom = Q2 + x * mW**2
    denom = np.where(denom == 0, np.finfo(float).tiny, denom)
    term_log1 = np.log((Q2 + x * mW**2) / (mW**2))
    term_log2 = np.log(((1.0 - x)**2) / (1.0 + x * (1.0 - x)**2))
    term_frac = x * mW**2 / denom
    term_last = 1.0 / (1.0 + x * (1.0 - x)**2) - 1.0
    bracket = term_log1 + term_log2 + term_frac + term_last
    return (alpha2 / (8.0 * np.pi)) * th * P * bracket


# -----------------------------------------------------------------------------
# Placeholders for user-supplied functions
# -----------------------------------------------------------------------------
# Replace these later with your model-dependent definitions:
def P_Vff(x): raise NotImplementedError("Define your P_Vff(x)")
def P_Vpm_fL(x): raise NotImplementedError("Define your P_Vpm_fL(x)")
def P_Vpm_fR(x): raise NotImplementedError("Define your P_Vpm_fR(x)")
def I_VVf(x): raise NotImplementedError("Define your I_VVf(x)")
def I_ffff(x): raise NotImplementedError("Define your I_ffff(x)")
def I_Vfff(x): raise NotImplementedError("Define your I_Vfff(x)")



Loading the PDFs from the LHAPDF files:

In [ ]:
def load_pdf_grid(path):
    with open(path) as f:
        lines = f.readlines()

    # Clean and strip lines
    lines = [l.strip() for l in lines if l.strip()]

    # Basic metadata
    pdf_type = lines[0].split(":")[1].strip()
    fmt = lines[1].split(":")[1].strip()

    # Extract grids
    x_grid = np.array(lines[3].split(), dtype=float)
    q_grid = np.array(lines[4].split(), dtype=float)

    # Labels and metadata
    labels = lines[5].split()
    pdg_ids = lines[6].split()
    polarizations = lines[7].split()

    nx = len(x_grid)
    nq = len(q_grid)
    n_flavs = len(labels)

    # The remaining lines are PDF values
    data_lines = lines[8:]
    assert len(data_lines) == nx * nq, f"Expected {nx*nq} data lines, got {len(data_lines)}"

    # Load as a big array: shape = (nq, nx, n_flavs)
    data = np.zeros((nq, nx, n_flavs), dtype=float)

    for iq in range(nq):
        for ix in range(nx):
            line_index = iq * nx + ix
            row = np.array(data_lines[line_index].split(), dtype=float)
            data[iq, ix, :] = row

    # Build interpolators for each flavor
    pdfs = {}
    for i, label in enumerate(labels):
        interp = RegularGridInterpolator(
            (np.log(x_grid), np.log(q_grid)),
            np.log(np.clip(data[:, :, i].T, 1e-30, None)),  # log–log interpolation
            bounds_error=False,
            fill_value=None
        )
        pdfs[label] = lambda x, q, interp=interp: np.exp(interp((np.log(x), np.log(q))))

    return {
        "x": x_grid,
        "Q": q_grid,
        "labels": labels,
        "pdg_ids": pdg_ids,
        "polarizations": polarizations,
        "pdfs": pdfs,
        "pdf_type": pdf_type,
        "format": fmt
    }


This function can be used like this:

In [ ]:
grid = load_pdf_grid("lePDFs/LePDF_mu_6FS_0000.dat")

# Example: get uL PDF from a muon, at x=0.01, Q=100 GeV
lepPDF = grid["pdfs"]["uL"](0.01, 100.0)
print(lepPDF)


For the CP conjugated:

In [ ]:
ParticleList6FS = ["eL","eR","nue","muL","muR","numu","taL","taR","nuta","eLb","eRb","nueb", "muLb", "muRb", "numub", "taLb", "taRb", "nutab", "dL", "dR", "uL", "uR", "sL", "sR", "cL", "cR", "bL", "bR", "tL","tR",
                   "dLb", "dRb", "uLb", "uRb", "sLb", "sRb", "cLb", "cRb", "bLb", "bRb", "tLb", "tRb", 
                   "gp", "gm", "gap", "gam", "Zp", "Zm","ZL", "Zgap", "Zgam", "Wpp", "Wpm", "WpL", "Wmp", "Wmm", "WmL", "h", "hZL"]

ParticleList6FSCP = ["eLb","eRb", "nueb", "muLb", "muRb", "numub", "taLb", "taRb","nutab", "eL", "eR", "nue", "muL", "muR", "numu","taL","taR","nuta", "dLb", "dRb", "uLb", "uRb", "sLb", "sRb", "cLb", "cRb", "bLb", "bRb", "tLb", "tRb",
                     "dL", "dR", "uL", "uR", "sL", "sR", "cL", "cR", "bL", "bR", "tL", "tR", 
                     "gm", "gp", "gam", "gap", "Zm", "Zp", "ZL", "Zgam", "Zgap", "Wmm", "Wmp", "WmL", "Wpm", "Wpp", "WpL", "h", "hZL"]

def CPconjugate(particle):
    if particle in ParticleList6FS:
        index = ParticleList6FS.index(particle)
        return ParticleList6FSCP[index]
    else:
        raise ValueError(f"Particle '{particle}' not found in either list.")

# The CP conjugated example: get uL PDF from an anti-muon, at x=0.01, Q=100 GeV    
lepbarPDF = grid["pdfs"][CPconjugate("uL")](0.01, 100.0)
print(lepbarPDF)
